# Population as a raster grid

Until now, all the data you were working with were tables. However, not
everything is a table. Raster data are not that common in social
geography, but spatial data science is full of it, from satellite
imagery to population grids. In this session, you will learn how to work
with spatial raster data in Python and how to link raster to vector
using the ecosystem around the `xarray` package.

## Arrays and their many dimensions

Raster data are represented as arrays. Those can take many forms and
shapes. You already know `pandas` data structures, so let’s start with
those.

A `pandas.Series` is a 1-dimensional array with an index. A typical
array contains values of the same data type (e.g. `float` numbers), as
does a typical `Series`.

When it comes to geospatial raster data, one dimension is not enough.
Even the most basic raster, something like a [digital terrain
model](https://en.wikipedia.org/wiki/Digital_elevation_model) (DTM),
requires two dimensions. One represents longitude (or x when projected),
while the other latitude (or y), resulting in a 2-dimensional array.

But you don’t have to stop there. Take a typical satellite image. The
longitude and latitude dimensions are still present, but you have
different bands representing blue, green, red and often near-infra-red
frequencies, resulting in a 3-dimensional array (`lon`, `lat`, `band`).
Throw in time, and you’re now dealing with a 4-dimensional array (`lon`,
`lat`, `band`, `time`).

All these use cases fall under the umbrella of N-dimensional array
handling covered by the `xarray` package. Whereas a `pandas.Series` is a
1-dimensional array with an index, `xarray.DataArray` is an
N-dimensional array with N indexes. Combining multiple `Series` gives
you a `pandas.DataFrame`, where each column can have a different data
type (e.g. one numbers, other names). Combining multiple
`xarray.DataArray`s gives you a `xarray.Dataset`, where each array can
have a different data type. There’s a lot of similarity between `pandas`
and `xarray`, but also some differences.

Let’s read some raster and explore `xarray` objects in practice.

In [2]:
import datashader as ds
import geopandas as gpd
import rioxarray
import xarray as xr
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import xvec

## Population grids

Today, you will be working with the data from the [Global Human
Settlement Layer](https://ghsl.jrc.ec.europa.eu/datasets.php) (GHSL)
developed by the Joint Research Centre of the European Commission.
Unlike in all previous hands-on sessions, the data is not pre-processed
and you could read it directly from the open data repository. However,
since that seems to be a bit unstable lately, use the copy stored as
part of this course.

The first layer you will open is a population grid. GHSL covers the
whole world divided into a set of tiles, each covering an area of 1,000
by 1,000 km at a resolution of 100m per pixel. The link below points to
a single tile[1] covering most of Eastern Europe.

[1] See the distribution of tiles in the [data
repository](https://ghsl.jrc.ec.europa.eu/download.php?ds=pop).

In [3]:
pop_url = (
    "https://martinfleischmann.net/sds/raster_data/data/"
    "GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip"
      )
pop_url

'https://martinfleischmann.net/sds/raster_data/data/GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip'

> **Original data**
>
> You can, alternatively, try reading the original data directly using
> this URL:
>
> ``` py
> pop_url = (
>     "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/"
>     "GHS_POP_GLOBE_R2023A/GHS_POP_E2030_GLOBE_R2023A_54009_100/"
>     "V1-0/tiles/GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip"
>       )
> ```

The `pop_url` points to a ZIP file. Within that ZIP file is a
[GeoTIFF](https://en.wikipedia.org/wiki/GeoTIFF) containing the actual
raster. There is often no need to download and unzip the file as there’s
a good chance you can read it directly.

### Reading rasters with `rioxarray`

`xarray`, like `pandas` is an agnostic library. It is designed for
N-dimensional arrays but not necessarily geospatial arrays (although
that is often the case…). It means that by default, it is not able to
read geospatial file formats like GeoTIFF. That is where `rioxarray`
comes in. It comes with the support of the usual geo-specific things
like specific file formats or CRS.

In [4]:
p = f"zip+{pop_url}!GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.tif"
population = rioxarray.open_rasterio(p, masked=True)
population

<xarray.DataArray (band: 1, y: 10000, x: 10000)> Size: 800MB
[100000000 values with dtype=float64]
Coordinates:
 * band (band) int64 8B 1
 * x (x) float64 80kB 9.59e+05 9.592e+05 ... 1.959e+06 1.959e+06
 * y (y) float64 80kB 6e+06 6e+06 6e+06 6e+06 ... 5e+06 5e+06 5e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray band : 1 y : 10000 x : 10000 ... [100000000 values with dtype=float64] Coordinates: (4) band (band) int64 1 array([1]) x (x) float64 9.59e+05 9.592e+05 ... 1.959e+06 array([ 959050., 959150., 959250., ..., 1958750., 1958850., 1958950.]) y (y) float64 6e+06 6e+06 6e+06 ... 5e+06 5e+06 array([5999950., 5999850., 5999750., ..., 5000250., 5000150., 5000050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 959000.0 100.0 0.0 6000000.0 0.0 -100.0 array(0) Indexes: (3) band PandasIndex PandasIndex(Index([1], dtype='int64', name='band')) x PandasIndex PandasIndex(Index([ 959050.0, 959150.0, 959250.0, 959350.0, 959450.0, 959550.0,
 959650.0, 959750.0, 959850.0, 959950.0,
 ...
 1958050.0, 1958150.0, 1958250.0, 1958350.0, 1958450.0, 1958550.0,
 1958650.0, 1958750.0, 1958850.0, 1958950.0],
 dtype='float64', name='x', length=10000)) y PandasIndex PandasIndex(Index([5999950.0, 5999850.0, 5999750.0, 5999650.0, 5999550.0, 5999450.0,
 5999350.0, 5999250.0, 5999150.0, 5999050.0,
 ...
 5000950.0, 5000850.0, 5000750.0, 5000650.0, 5000550.0, 5000450.0,
 5000350.0, 5000250.0, 5000150.0, 5000050.0],
 dtype='float64', name='y', length=10000)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

Above, you can see the representation of the population grid as a
`DataArray`. It has three dimensions (`"band"`, `"x"`, `"y"`) with a
resolution 1x10,000x10,000 and values as `float`.

`rioxarray` gives you a handy `.rio` accessor on `xarray` objects,
allowing you to access geospatial-specific tools. Like retrieval of CRS.

In [5]:
population.rio.crs

CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

Or the extent of the raster (in the CRS shown above).

In [6]:
population.rio.bounds()

(959000.0, 5000000.0, 1959000.0, 6000000.0)

The missing, masked data can be represented as as specific value,
especially when dealing with integer arrays. You can check which one:

In [7]:
population.rio.nodata

nan

### Plotting with `datashader`

Plotting a raster with a resolution of 10,000x10,000 pixels can be
tricky. Often, the resolution is even larger than that. The best way to
plot is to resample the data to a smaller resolution that better fits
the screen. A handy tool that can do that quickly is
[`datashader`](https://datashader.org). Let’s use it to plot the array
as 600x600 pixels.

In [8]:
canvas = ds.Canvas(plot_width=600, plot_height=600)
agg = canvas.raster(population.where(population>0).sel(band=1))
agg

<xarray.DataArray (y: 600, x: 600)> Size: 3MB
array([[4.21513869, 3.52179019, 6.29294423, ..., 1.66938128, 0.82322911,
 nan],
 [1.04041781, 1.57566844, 4.50548627, ..., nan, 0.15449866,
 nan],
 [ nan, 3.18057789, 2.21502486, ..., nan, nan,
 nan],
 ...,
 [ nan, nan, nan, ..., 4.05967611, 1.9045163 ,
 3.70262405],
 [ nan, nan, nan, ..., 7.2545469 , 6.44937184,
 1.5507871 ],
 [ nan, nan, nan, ..., 0.76526973, 1.16737383,
 0.21985548]])
Coordinates:
 * x (x) float64 5kB 9.598e+05 9.615e+05 ... 1.957e+06 1.958e+06
 * y (y) float64 5kB 5.999e+06 5.998e+06 ... 5.002e+06 5.001e+06
Attributes:
 res: 100.0
 x_range: (959000.0, 1959000.0)
 y_range: (5000000.0, 6000000.0) xarray.DataArray y : 600 x : 600 4.215 3.522 6.293 2.667 2.879 2.336 ... 1.52 1.683 0.7653 1.167 0.2199 array([[4.21513869, 3.52179019, 6.29294423, ..., 1.66938128, 0.82322911,
 nan],
 [1.04041781, 1.57566844, 4.50548627, ..., nan, 0.15449866,
 nan],
 [ nan, 3.18057789, 2.21502486, ..., nan, nan,
 nan],
 ...,
 [ nan, nan, nan, ..., 4.05967611, 1.9045163 ,
 3.70262405],
 [ nan, nan, nan, ..., 7.2545469 , 6.44937184,
 1.5507871 ],
 [ nan, nan, nan, ..., 0.76526973, 1.16737383,
 0.21985548]]) Coordinates: (2) x (x) float64 9.598e+05 9.615e+05 ... 1.958e+06 array([ 959833.333333, 961500. , 963166.666667, ..., 1954833.333333,
 1956500. , 1958166.666667]) y (y) float64 5.999e+06 5.998e+06 ... 5.001e+06 array([5999166.666667, 5997500. , 5995833.333333, ..., 5004166.666667,
 5002500. , 5000833.333333]) Indexes: (2) x PandasIndex PandasIndex(Index([ 959833.3333333334, 961500.0, 963166.6666666667,
 964833.3333333334, 966500.0, 968166.6666666667,
 969833.3333333334, 971500.0, 973166.6666666667,
 974833.3333333334,
 ...
 1943166.666666667, 1944833.3333333337, 1946500.0000000002,
 1948166.666666667, 1949833.3333333337, 1951500.0000000002,
 1953166.666666667, 1954833.3333333337, 1956500.0000000002,
 1958166.666666667],
 dtype='float64', name='x', length=600)) y PandasIndex PandasIndex(Index([5999166.666666666, 5997500.0, 5995833.333333333,
 5994166.666666666, 5992500.0, 5990833.333333333,
 5989166.666666666, 5987500.0, 5985833.333333333,
 5984166.666666666,
 ...
 5015833.333333333, 5014166.666666666, 5012500.0,
 5010833.333333333, 5009166.666666666, 5007500.0,
 5005833.333333333, 5004166.666666666, 5002500.0,
 5000833.333333333],
 dtype='float64', name='y', length=600)) Attributes: (3) res : 100.0 x_range : (959000.0, 1959000.0) y_range : (5000000.0, 6000000.0)

You can see that the result is a new `xarray.DataArray` with a
resolution 600x600. The built-in matplotlib-based plotting can easily
handle that.

In [9]:
_ = agg.plot()

## Clipping based on geometry

Daling with large rasters is often impractical if you are interested in
a small subset, for example, representing a single city.

### Functional Urban Areas

In this case, you may want to work only with the data covering Budapest,
Hungary, defined by its [functional urban
area](https://ghsl.jrc.ec.europa.eu/ghs_fua.php) (FUA), available as
another data product on GHSL. FUAs are available as a single GeoPackage
with vector geometries.

In [10]:
fua_url = (
    "https://martinfleischmann.net/sds/raster_data/data/"
    "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.zip"
)
p = f"zip+{fua_url}!GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
fuas = gpd.read_file(p)
budapest = fuas.query("eFUA_name == 'Budapest'")
budapest.explore()

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_db9c4f79cdd0a03453d16517ef0efdba {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>
 
 
 <style>
 .foliumtooltip {
 
 }
 .foliumtooltip table{
 margin: auto;
 }
 .foliumtooltip tr{
 text-align: left;
 }
 .foliumtooltip th{
 padding: 2px; padding-right: 8px;
 }
 </style>
 
</head>
<body>
 
 
 <div class="folium-map" id="map_db9c4f79cdd0a03453d16517ef0efdba" ></div>
 
</body>
<script>
 
 
 var map_db9c4f79cdd0a03453d16517ef0efdba = L.map(
 "map_db9c4f79cdd0a03453d16517ef0efdba",
 {
 center: [47.47044485323768, 19.212935175082336],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 }
 );
 L.control.scale().addTo(map_db9c4f79cdd0a03453d16517ef0efdba);

 

 
 
 var tile_layer_a38c8d3828e72f2ea62c1bec36600d8f = L.tileLayer(
 "https://tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors", "detectRetina": false, "maxNativeZoom": 19, "maxZoom": 19, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 );
 
 
 tile_layer_a38c8d3828e72f2ea62c1bec36600d8f.addTo(map_db9c4f79cdd0a03453d16517ef0efdba);
 
 
 map_db9c4f79cdd0a03453d16517ef0efdba.fitBounds(
 [[47.07677314279965, 18.712248503838712], [47.86411656367571, 19.713621846325957]],
 {}
 );
 
 
 function geo_json_e0b9fd83575981008a2f51fb39c0b4d4_styler(feature) {
 switch(feature.id) {
 default:
 return {"fillOpacity": 0.5, "weight": 2};
 }
 }
 function geo_json_e0b9fd83575981008a2f51fb39c0b4d4_highlighter(feature) {
 switch(feature.id) {
 default:
 return {"fillOpacity": 0.75};
 }
 }
 function geo_json_e0b9fd83575981008a2f51fb39c0b4d4_pointToLayer(feature, latlng) {
 var opts = {"bubblingMouseEvents": true, "color": "#3388ff", "dashArray": null, "dashOffset": null, "fill": true, "fillColor": "#3388ff", "fillOpacity": 0.2, "fillRule": "evenodd", "lineCap": "round", "lineJoin": "round", "opacity": 1.0, "radius": 2, "stroke": true, "weight": 3};
 
 let style = geo_json_e0b9fd83575981008a2f51fb39c0b4d4_styler(feature)
 Object.assign(opts, style)
 
 return new L.CircleMarker(latlng, opts)
 }

 function geo_json_e0b9fd83575981008a2f51fb39c0b4d4_onEachFeature(feature, layer) {
 layer.on({
 mouseout: function(e) {
 if(typeof e.target.setStyle === "function"){
 geo_json_e0b9fd83575

> **Original data**
>
> You can, alternatively, try reading the original data directly using
> this URL:
>
> ``` py
> fua_url = (
>     "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/"
>     "GHS_FUA_UCDB2015_GLOBE_R2019A/V1-0/"
>     "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.zip"
> )
> ```

If you want to clip the `population` raster to the extent of Budapest
FUA, you can use the `clip` method from the `rioxarray` extension of
`xarray`.

In [11]:
population_bud = population.rio.clip(
    budapest.to_crs(population.rio.crs).geometry
)
population_bud

<xarray.DataArray (band: 1, y: 840, x: 830)> Size: 6MB
array([[[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
 * band (band) int64 8B 1
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray band : 1 y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]]) Coordinates: (4) band (band) int64 1 array([1]) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (3) band PandasIndex PandasIndex(Index([1], dtype='int64', name='band')) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

The raster is no longer 10,000x10,000 pixels but only 840x830, covering
the extent of Budapest FUA. You can easily check that by plotting the
clipped array.

In [12]:
_ = population_bud.plot()

## Array manipulation

While this is technically a 3-dimensional array, the dimension `"band"`
has only one value. Normally, you would get a 2-dimensional array
representing a selected band using the `.sel()` method.

In [13]:
population_bud.sel(band=1)

<xarray.DataArray (y: 840, x: 830)> Size: 6MB
array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
 band int64 8B 1
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]) Coordinates: (4) band () int64 1 array(1) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (2) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

But if you have only one band, you can *squeeze* the array and get rid
of that dimension that is not needed.

In [14]:
population_bud = population_bud.drop_vars("band").squeeze()
population_bud

<xarray.DataArray (y: 840, x: 830)> Size: 6MB
array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]) Coordinates: (3) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (2) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

Now a lot what you know from `pandas` works equally in `xarray`. Getting
the minimum:

In [15]:
population_bud.min()

<xarray.DataArray ()> Size: 8B
array(0.)
Coordinates:
 spatial_ref int64 8B 0 xarray.DataArray 0.0 array(0.) Coordinates: (1) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (0) Attributes: (0)

As expected, there are some cells with no inhabitants.

In [16]:
population_bud.max()

<xarray.DataArray ()> Size: 8B
array(603.90405273)
Coordinates:
 spatial_ref int64 8B 0 xarray.DataArray 603.9 array(603.90405273) Coordinates: (1) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (0) Attributes: (0)

The densest cell, on the other hand, has more than 600 people per
hectare.

In [17]:
population_bud.mean()

<xarray.DataArray ()> Size: 8B
array(6.77593517)
Coordinates:
 spatial_ref int64 8B 0 xarray.DataArray 6.776 array(6.77593517) Coordinates: (1) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (0) Attributes: (0)

Mean is, however, only below 7.

In [18]:
population_bud.median()

<xarray.DataArray ()> Size: 8B
array(0.)
Coordinates:
 spatial_ref int64 8B 0 xarray.DataArray 0.0 array(0.) Coordinates: (1) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (0) Attributes: (0)

While the median is 0, there are a lot of cells with 0.

> **DataArray vs scalar**
>
> Notice that `xarray` always returns another `DataArray` even with a
> single value. If you want to get that scalar value, you can use
> `.item()`.
>
> ``` python
> population_bud.mean().item()
> ```
>
>     6.775935172506021

You can plot the distribution of values across the array.

In [20]:
_ = population_bud.plot.hist(bins=100)

Indeed, there are a lot of zeros. Let’s filter them out and check the
distribution again.

In [21]:
_ = population_bud.where(population_bud>0).plot.hist(bins=100)

As with many observations in urban areas, this follows a power-law-like
distribution with a lot of observations with tiny values and only a few
with large ones.

## Array operations

Let’s assume that you want to normalise population counts by the
built-up volume, which is available as another GHSL product. This time,
on a grid again.

In [22]:
volume_url = (
    "https://martinfleischmann.net/sds/raster_data/data/"
    "GHS_BUILT_V_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip"
)
volume_url

'https://martinfleischmann.net/sds/raster_data/data/GHS_BUILT_V_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip'

> **Backup data**
>
> You can, alternatively, try reading the original data directly using
> this URL:
>
> ``` py
> volume_url = (
>     "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/"
>     "GHS_BUILT_V_GLOBE_R2023A/GHS_BUILT_V_E2030_GLOBE_R2023A_54009_100/V1-0/tiles/"
>     "GHS_BUILT_V_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.zip"
> )
> ```

All work the same as before. You read the GeoTIFF as a `DataArray`.

In [23]:
p = f"zip+{volume_url}!GHS_BUILT_V_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C20.tif"
built_up = rioxarray.open_rasterio(p, masked=True).drop_vars("band").squeeze()
built_up

<xarray.DataArray (y: 10000, x: 10000)> Size: 800MB
[100000000 values with dtype=float64]
Coordinates:
 * x (x) float64 80kB 9.59e+05 9.592e+05 ... 1.959e+06 1.959e+06
 * y (y) float64 80kB 6e+06 6e+06 6e+06 6e+06 ... 5e+06 5e+06 5e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray y : 10000 x : 10000 ... [100000000 values with dtype=float64] Coordinates: (3) x (x) float64 9.59e+05 9.592e+05 ... 1.959e+06 array([ 959050., 959150., 959250., ..., 1958750., 1958850., 1958950.]) y (y) float64 6e+06 6e+06 6e+06 ... 5e+06 5e+06 array([5999950., 5999850., 5999750., ..., 5000250., 5000150., 5000050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 959000.0 100.0 0.0 6000000.0 0.0 -100.0 array(0) Indexes: (2) x PandasIndex PandasIndex(Index([ 959050.0, 959150.0, 959250.0, 959350.0, 959450.0, 959550.0,
 959650.0, 959750.0, 959850.0, 959950.0,
 ...
 1958050.0, 1958150.0, 1958250.0, 1958350.0, 1958450.0, 1958550.0,
 1958650.0, 1958750.0, 1958850.0, 1958950.0],
 dtype='float64', name='x', length=10000)) y PandasIndex PandasIndex(Index([5999950.0, 5999850.0, 5999750.0, 5999650.0, 5999550.0, 5999450.0,
 5999350.0, 5999250.0, 5999150.0, 5999050.0,
 ...
 5000950.0, 5000850.0, 5000750.0, 5000650.0, 5000550.0, 5000450.0,
 5000350.0, 5000250.0, 5000150.0, 5000050.0],
 dtype='float64', name='y', length=10000)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

And clip it to the same extent.

In [24]:
built_up_bud = built_up.rio.clip(budapest.to_crs(built_up.rio.crs).geometry)
built_up_bud

<xarray.DataArray (y: 840, x: 830)> Size: 6MB
array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
Attributes:
 AREA_OR_POINT: Area
 scale_factor: 1.0
 add_offset: 0.0 xarray.DataArray y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]) Coordinates: (3) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (2) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (3) AREA_OR_POINT : Area scale_factor : 1.0 add_offset : 0.0

You can quickly check what it looks like.

In [25]:
_ = built_up_bud.plot(cmap="magma_r")

The two grids are aligned, meaning that pixels with the same coordinates
represent the same area. This allows us to directly perform array
algebra. Again, you know this from `pandas`.

In [26]:
pop_density = population_bud /  built_up_bud
pop_density

<xarray.DataArray (y: 840, x: 830)> Size: 6MB
array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0 xarray.DataArray y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]) Coordinates: (3) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) Indexes: (2) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (0)

The result is a new array that inherits spatial information
(`spatial_ref`) but contains newly computed values.

In [27]:
_ = pop_density.plot(cmap="cividis_r")

The resulting array can then be saved to a GeoTIFF using `rioxarray`.

In [28]:
pop_density.rio.to_raster("population_density.tif")

## Extracting values for locations

A common need is to extract values from raster data for a specific
location of interest. That is a first type of interaction between raster
and vector data (points in this case). To illustrate the use case,
create a set of random points covering the area of `budapest`.

In [29]:
locations = budapest.sample_points(1000).explode(ignore_index=True)
locations.head()

0    POINT (1467057.411 5633041.746)
1    POINT (1467289.991 5633881.564)
2    POINT (1467396.432 5628056.263)
3    POINT (1468318.618 5629152.178)
4    POINT (1468416.314 5629482.562)
Name: sampled_points, dtype: geometry

Check how the sample looks on a map.

In [30]:
locations.explore()

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_adc8f33d7d543dcb566cde3ab0273386 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>
 
 
 <style>
 .foliumtooltip {
 
 }
 .foliumtooltip table{
 margin: auto;
 }
 .foliumtooltip tr{
 text-align: left;
 }
 .foliumtooltip th{
 padding: 2px; padding-right: 8px;
 }
 </style>
 
</head>
<body>
 
 
 <div class="folium-map" id="map_adc8f33d7d543dcb566cde3ab0273386" ></div>
 
</body>
<script>
 
 
 var map_adc8f33d7d543dcb566cde3ab0273386 = L.map(
 "map_adc8f33d7d543dcb566cde3ab0273386",
 {
 center: [47.4675328657236, 19.20684158996545],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 }
 );
 L.control.scale().addTo(map_adc8f33d7d543dcb566cde3ab0273386);

 

 
 
 var tile_layer_16727791d2af64f93917ccd310cd04ca = L.tileLayer(
 "https://tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors", "detectRetina": false, "maxNativeZoom": 19, "maxZoom": 19, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 );
 
 
 tile_layer_16727791d2af64f93917ccd310cd04ca.addTo(map_adc8f33d7d543dcb566cde3ab0273386);
 
 
 map_adc8f33d7d543dcb566cde3ab0273386.fitBounds(
 [[47.0864944787123, 18.721793177729182], [47.84857125273489, 19.691890002201713]],
 {}
 );
 
 
 function geo_json_506e7605de8433b331e9a69a274deabc_styler(feature) {
 switch(feature.id) {
 default:
 return {"fillOpacity": 0.5, "weight": 2};
 }
 }
 function geo_json_506e7605de8433b331e9a69a274deabc_highlighter(feature) {
 switch(feature.id) {
 default:
 return {"fillOpacity": 0.75};
 }
 }
 function geo_json_506e7605de8433b331e9a69a274deabc_pointToLayer(feature, latlng) {
 var opts = {"bubblingMouseEvents": true, "color": "#3388ff", "dashArray": null, "dashOffset": null, "fill": true, "fillColor": "#3388ff", "fillOpacity": 0.2, "fillRule": "evenodd", "lineCap": "round", "lineJoin": "round", "opacity": 1.0, "radius": 2, "stroke": true, "weight": 3};
 
 let style = geo_json_506e7605de8433b331e9a69a274deabc_styler(feature)
 Object.assign(opts, style)
 
 return new L.CircleMarker(latlng, opts)
 }

 function geo_json_506e7605de8433b331e9a69a274deabc_onEachFeature(feature, layer) {
 layer.on({
 mouseout: function(e) {
 if(typeof e.target.setStyle === "function"){
 geo_json_506e7605de8433

> **Random sampling and reproducibility**
>
> The points sampled from `budapest` will be different every time you
> run the `sample_points()` method. If you want to fix the result, you
> can pass a *seed* value to a random number generator as `rng=42`. With
> the same seed value, the result will be always the same. This is
> useful, especially if you are interested in the reproducibility of
> your code.

The `xarray` ecosystem offers many ways of extracting point values.
Below, you will use the implementation from the `xvec` package. Create a
new `xarray.DataArray` with all three arrays you created so far to see
the benefits of using `xvec` below.

In [31]:
bud_cube = xr.concat(
    [pop_density, population_bud, built_up_bud],
    dim=pd.Index(
        ["density", "population", "built-up volume"],
        name="measurement",
    )
)
bud_cube

<xarray.DataArray (measurement: 3, y: 840, x: 830)> Size: 17MB
array([[[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]],

 [[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]],

 [[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
 * measurement (measurement) object 24B 'density' ... 'built-up volume' xarray.DataArray measurement : 3 y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]],

 [[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]],

 [[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]]) Coordinates: (4) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) measurement (measurement) object 'density' ... 'built-up volume' array(['density', 'population', 'built-up volume'], dtype=object) Indexes: (3) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) measur

The resulting `DataArray` is 3-dimensional, compared to 2-dimensional
arrays used before. Apart from `x` and `y`, you now have `measurement`
as well. Using the new index created above, you can use the `sel()`
method to get the original arrays.

In [32]:
bud_cube.sel(measurement="density")

<xarray.DataArray (y: 840, x: 830)> Size: 6MB
array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
 * x (x) float64 7kB 1.465e+06 1.465e+06 ... 1.548e+06 1.548e+06
 * y (y) float64 7kB 5.648e+06 5.648e+06 ... 5.564e+06 5.564e+06
 spatial_ref int64 8B 0
 measurement <U7 28B 'density' xarray.DataArray y : 840 x : 830 nan nan nan nan nan nan nan nan ... nan nan nan nan nan nan nan nan array([[nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 ...,
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan],
 [nan, nan, nan, ..., nan, nan, nan]]) Coordinates: (4) x (x) float64 1.465e+06 1.465e+06 ... 1.548e+06 axis : X long_name : x coordinate of projection standard_name : projection_x_coordinate units : metre array([1465050., 1465150., 1465250., ..., 1547750., 1547850., 1547950.]) y (y) float64 5.648e+06 5.648e+06 ... 5.564e+06 axis : Y long_name : y coordinate of projection standard_name : projection_y_coordinate units : metre array([5647950., 5647850., 5647750., ..., 5564250., 5564150., 5564050.]) spatial_ref () int64 0 crs_wkt : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] spatial_ref : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] GeoTransform : 1465000.0 100.0 0.0 5648000.0 0.0 -100.0 array(0) measurement () <U7 'density' array('density', dtype='<U7') Indexes: (2) x PandasIndex PandasIndex(Index([1465050.0, 1465150.0, 1465250.0, 1465350.0, 1465450.0, 1465550.0,
 1465650.0, 1465750.0, 1465850.0, 1465950.0,
 ...
 1547050.0, 1547150.0, 1547250.0, 1547350.0, 1547450.0, 1547550.0,
 1547650.0, 1547750.0, 1547850.0, 1547950.0],
 dtype='float64', name='x', length=830)) y PandasIndex PandasIndex(Index([5647950.0, 5647850.0, 5647750.0, 5647650.0, 5647550.0, 5647450.0,
 5647350.0, 5647250.0, 5647150.0, 5647050.0,
 ...
 5564950.0, 5564850.0, 5564750.0, 5564650.0, 5564550.0, 5564450.0,
 5564350.0, 5564250.0, 5564150.0, 5564050.0],
 dtype='float64', name='y', length=840)) Attributes: (0)

Now it is time to take this *cube* and create another based on your
points. That can be done using the `.xvec` accessor and its
`extract_points` method.

In [33]:
vector_cube = bud_cube.drop_vars("spatial_ref").xvec.extract_points(
    points=locations.geometry,
    x_coords="x",
    y_coords="y",
)
vector_cube

<xarray.DataArray (measurement: 3, geometry: 1000)> Size: 24kB
array([[1.69309008e-03, 1.11217133e-03, nan, ...,
 nan, 1.67070977e-03, nan],
 [3.85584335e+01, 2.02971268e+00, 0.00000000e+00, ...,
 0.00000000e+00, 1.37198687e+01, 0.00000000e+00],
 [2.27740000e+04, 1.82500000e+03, 0.00000000e+00, ...,
 0.00000000e+00, 8.21200000e+03, 0.00000000e+00]])
Coordinates:
 * measurement (measurement) object 24B 'density' ... 'built-up volume'
 * geometry (geometry) object 8kB POINT (1467057.411131141 5633041.74624...
Indexes:
 geometry GeometryIndex (crs=ESRI:54009) xarray.DataArray measurement : 3 geometry : 1000 0.001693 0.001112 nan nan nan nan ... 0.0 0.0 0.0 0.0 8.212e+03 0.0 array([[1.69309008e-03, 1.11217133e-03, nan, ...,
 nan, 1.67070977e-03, nan],
 [3.85584335e+01, 2.02971268e+00, 0.00000000e+00, ...,
 0.00000000e+00, 1.37198687e+01, 0.00000000e+00],
 [2.27740000e+04, 1.82500000e+03, 0.00000000e+00, ...,
 0.00000000e+00, 8.21200000e+03, 0.00000000e+00]]) Coordinates: (2) measurement (measurement) object 'density' ... 'built-up volume' array(['density', 'population', 'built-up volume'], dtype=object) geometry (geometry) object POINT (1467057.411131141 5633041... crs : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54009"]] array([<POINT (1467057.411 5633041.746)>, <POINT (1467289.991 5633881.564)>,
 <POINT (1467396.432 5628056.263)>, ..., <POINT (1545915.8 5586927.296)>,
 <POINT (1546603.115 5603215.433)>, <POINT (1546836.809 5596484.268)>],
 dtype=object) Indexes: (2) measurement PandasIndex PandasIndex(Index(['density', 'population', 'built-up volume'], dtype='object', name='measurement')) geometry GeometryIndex (crs=ESRI:54009) GeometryIndex(
 [<POINT (1467057.411 5633041.746)>
 <POINT (1467289.991 5633881.564)>
 <POINT (1467396.432 5628056.263)>
 <POINT (1468318.618 5629152.178)>
 ...
 <POINT (1545872.605 5587010.617)>
 <POINT (1545915.8 5586927.296)>
 <POINT (1546603.115 5603215.433)>
 <POINT (1546836.809 5596484.268)>],
 crs=ESRI:54009) Attributes: (0)

The resulting object is still a `DataArray` but a bit different. It is
no longer 3-dimensional, although all dimensions of interest
(`'density', 'population', 'built-up volume'`) are still there, but
2-dimensional. One dimension is `measurement`, and the other is
`geometry`, containing the points of interest. With `xvec`, the spatial
dimension is reduced, but the remaining dimensionality of the original
array is preserved.

You can then convert the data into a `geopandas.GeoDataFrame` and work
with it as usual.

In [34]:
location_data = vector_cube.xvec.to_geopandas()
location_data.head()

Check the result on a map to verify that all worked as expected.

In [35]:
location_data.explore("density", cmap="cividis_r", tiles="CartoDB Positron")

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_202d978ee050ec614af27013276dac30 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>
 
 
 <style>
 .foliumtooltip {
 
 }
 .foliumtooltip table{
 margin: auto;
 }
 .foliumtooltip tr{
 text-align: left;
 }
 .foliumtooltip th{
 padding: 2px; padding-right: 8px;
 }
 </style>
 
 <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
</head>
<body>
 
 
 <div class="folium-map" id="map_202d978ee050ec614af27013276dac30" ></div>
 
</body>
<script>
 
 
 var map_202d978ee050ec614af27013276dac30 = L.map(
 "map_202d978ee050ec614af27013276dac30",
 {
 center: [47.4675328657236, 19.20684158996545],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 }
 );
 L.control.scale().addTo(map_202d978ee050ec614af27013276dac30);

 

 
 
 var tile_layer_5ece496efefbe8402afff5a321190284 = L.tileLayer(
 "https://a.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
 {"attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"https://carto.com/attributions\"\u003eCARTO\u003c/a\u003e", "detectRetina": false, "maxZoom": 20, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 );
 
 
 tile_layer_5ece496efefbe8402afff5a321190284.addTo(map_202d978ee050ec614af27013276dac30);
 
 
 map_202d978ee050ec614af27013276dac30.fitBounds(
 [[47.0864944787123, 18.721793177729182], [47.84857125273489, 19.691890002201713]],
 {}
 );
 
 
 function geo_json_de16d2335864a355c77651615303f5a8_styler(feature) {
 switch(feature.id) {
 case "0": 
 return {"color": "#958f77", "fillColor": "#958f77", "fillOpacity": 0.5, "weight": 2};
 case "1": case "6": case "530": 
 return {"color": "#baac6d", "fillColor": "#baac6d", "fillOpacity": 0.5, "weight": 2};
 case "2": case "3": case "4": case "5": case "7": case "8": case "9": case "10": case "11": case "12": case "13": case "14": case "15": case "17": case "18": case "20": case "22": case "23": case "24": case "26": case "29": case "30": case "31": case "32": case "33": case "35": case "36": case "37": case "38": case "40": case "41": case "42": case "43": case "44": case "45": case "46": case "48": case "49": case "50": case "51": case "52": case "53": case "54": case "56": case "57": case "59": case "60": case "61": case 

> **Vector data cubes**
>
> The data structure `vector_cube` represents is called a vector data
> cube. It is a special case of an `xarray` N-dimensional object, where
> at least one dimension is indexed by geometries. See more in the [Xvec
> documentation](https://xvec.readthedocs.io/en/stable/intro.html).

## Zonal statistics

Another operation when working with rasters is the transfer of values
from an array to a set of polygons. This is called *zonal statistics*
and can be done in many ways, depending on the use case. In most cases,
one of the methods available in `xvec` should cover your specific needs.

### Downloading OpenStreetMap data

You may be interested in the average population density in individual
districts of Budapest. One option for getting the geometries
representing the districts is the
[OpenStreetMap](https://www.openstreetmap.org/). Everything you can see
on OpenStreetMap is downloadable. In Python, a recommended way (when not
doing large downloads) is the `osmnx` package (imported as `ox`). The
detailed explanation of `osmnx` is out of scope for this session, but if
you are interested in details, check the official [Getting
started](https://osmnx.readthedocs.io/en/stable/getting-started.html)
guide.

In [36]:
admin_level_9 = ox.features_from_place("Budapest", {"admin_level": "9"})
districts = admin_level_9[admin_level_9.geom_type == "Polygon"][
    ["name", "name:en", "geometry"]
]
districts["key"] = range(len(districts))
districts = districts.to_crs(pop_density.rio.crs)

### Plotting raster and vector together

Both `xarray` and `geopandas` can create `matplotlib` plots that can be
combined to see how the two overlap.

In [37]:
f, ax = plt.subplots()
pop_density.plot(ax=ax, cmap="cividis_r")
districts.plot(
    ax=ax, facecolor="none", edgecolor="red", linewidth=1, aspect=None
);

### Zonal statistics

Zonal statistics using `xvec` is as simple as point data extraction,
with the only difference that you can optionally specify the type of
aggregation you’d like to extract. By default, you get mean.

In [38]:
zonal_stats = bud_cube.drop_vars("spatial_ref").xvec.zonal_stats(
    geometry=districts.geometry,
    x_coords="x",
    y_coords="y",
)
zonal_stats

<xarray.DataArray (geometry: 23, measurement: 3)> Size: 552B
array([[1.93530445e-03, 3.05514687e+01, 1.93385830e+04],
 [1.55306987e-03, 1.75203519e+01, 1.24434552e+04],
 [2.97495556e-03, 3.44800938e+01, 1.28181432e+04],
 [2.28693771e-03, 2.57417575e+01, 1.14941986e+04],
 [2.57494673e-03, 7.65020268e+01, 3.08506154e+04],
 [2.23832080e-03, 2.30955540e+01, 1.07480293e+04],
 [2.64115677e-03, 4.61908980e+01, 2.03117370e+04],
 [3.38977582e-03, 5.35142704e+01, 1.77673289e+04],
 [1.47557004e-03, 5.79631789e+00, 5.26320947e+03],
 [2.10694012e-03, 2.73223607e+01, 1.37756218e+04],
 [2.15614865e-03, 1.65916271e+01, 7.80771904e+03],
 [2.56786753e-03, 6.79190265e+01, 2.69252700e+04],
 [2.67075964e-03, 5.70909834e+01, 2.16905234e+04],
 [2.13305661e-03, 5.39779792e+01, 3.07897010e+04],
 [2.13384957e-03, 2.67868574e+01, 1.63365941e+04],
 [1.95306643e-03, 2.28662318e+01, 1.21904109e+04],
 [2.78713051e-03, 1.05099081e+02, 4.07332861e+04],
 [2.46986944e-03, 3.14801382e+01, 1.33601201e+04],
 [2.62209009e-03, 7.51648089e+01, 3.01167759e+04],
 [3.09263579e-03, 1.20801957e+02, 4.18122526e+04],
 [2.89497635e-03, 1.67700965e+02, 6.04019202e+04],
 [4.16208535e-03, 2.86557806e+02, 6.89801196e+04],
 [2.16822023e-03, 1.06236529e+02, 5.05770076e+04]])
Coordinates:
 * measurement (measurement) object 24B 'density' ... 'built-up volume'
 * geometry (geometry) object 184B POLYGON ((1494534.3281858459 5597452....
 index (geometry) object 184B ('relation', 215618) ... ('relation',...
Indexes:
 geometry GeometryIndex (crs=ESRI:54009) xarray.DataArray geometry : 23 measurement : 3 0.001935 30.55 1.934e+04 0.001553 ... 0.002168 106.2 5.058e+04 array([[1.93530445e-03, 3.05514687e+01, 1.93385830e+04],
 [1.55306987e-03, 1.75203519e+01, 1.24434552e+04],
 [2.97495556e-03, 3.44800938e+01, 1.28181432e+04],
 [2.28693771e-03, 2.57417575e+01, 1.14941986e+04],
 [2.57494673e-03, 7.65020268e+01, 3.08506154e+04],
 [2.23832080e-03, 2.30955540e+01, 1.07480293e+04],
 [2.64115677e-03, 4.61908980e+01, 2.03117370e+04],
 [3.38977582e-03, 5.35142704e+01, 1.77673289e+04],
 [1.47557004e-03, 5.79631789e+00, 5.26320947e+03],
 [2.10694012e-03, 2.73223607e+01, 1.37756218e+04],
 [2.15614865e-03, 1.65916271e+01, 7.80771904e+03],
 [2.56786753e-03, 6.79190265e+01, 2.69252700e+04],
 [2.67075964e-03, 5.70909834e+01, 2.16905234e+04],
 [2.13305661e-03, 5.39779792e+01, 3.07897010e+04],
 [2.13384957e-03, 2.67868574e+01, 1.63365941e+04],
 [1.95306643e-03, 2.28662318e+01, 1.21904109e+04],
 [2.78713051e-03, 1.05099081e+02, 4.07332861e+04],
 [2.46986944e-03, 3.14801382e+01, 1.33601201e+04],
 [2.62209009e-03, 7.51648089e+01, 3.01167759e+04],
 [3.09263579e-03, 1.20801957e+02, 4.18122526e+04],
 [2.89497635e-03, 1.67700965e+02, 6.04019202e+04],
 [4.16208535e-03, 2.86557806e+02, 6.89801196e+04],
 [2.16822023e-03, 1.06236529e+02, 5.05770076e+04]]) Coordinates: (3) measurement (measurement) object 'density' ... 'built-up volume' array(['density', 'population', 'built-up volume'], dtype=object) geometry (geometry) object POLYGON ((1494534.3281858459 559... crs : PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] array([<POLYGON ((1494534.328 5597452.839, 1494709.464 5597483.689, 1495374.488 559...>,
 <POLYGON ((1489341.705 5597900.249, 1489344.48 5597918.588, 1489347.525 5597...>,
 <POLYGON ((1488148.017 5617092.544, 1488102.05 5617191.926, 1488138.549 5617...>,
 <POLYGON ((1484592.864 5613919.109, 1484887.294 5614074.835, 1484943.798 561...>,
 <POLYGON ((1493202.543 5608736.393, 1493201.024 5608743.668, 1493198.955 560...>,
 <POLYGON ((1485543.934 5612793.322, 1486597.575 5612075.431, 1486661.591 561...>,
 <POLYGON ((1489351.206 5606660.953, 1489351.382 5606668.8

The result is again a vector data cube. Though mean is not the optimal
aggregation method for `population`. Let’s refine the code above a bit
to get more useful aggregation.

In [39]:
zonal_stats = bud_cube.drop_vars("spatial_ref").xvec.zonal_stats(
    geometry=districts.geometry,
    x_coords="x",
    y_coords="y",
    stats=["mean", "sum", "median", "min", "max"],
)
zonal_stats

<xarray.DataArray (geometry: 23, zonal_statistics: 5, measurement: 3)> Size: 3kB
array([[[1.93530445e-03, 3.05514687e+01, 1.93385830e+04],
 [4.32734075e+00, 8.02281569e+04, 5.07831190e+07],
 [2.37775026e-03, 2.24578524e+01, 1.65745000e+04],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.31672247e-03, 2.00829544e+02, 1.49180000e+05]],

 [[1.55306987e-03, 1.75203519e+01, 1.24434552e+04],
 [4.03953473e+00, 5.88859028e+04, 4.18224530e+07],
 [1.70373389e-03, 9.53239155e+00, 7.51800000e+03],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.21869412e-03, 1.23922935e+02, 1.45769000e+05]],

 [[2.97495556e-03, 3.44800938e+01, 1.28181432e+04],
 [9.93932653e+00, 1.35334368e+05, 5.03112120e+07],
 [3.32938273e-03, 2.29527645e+01, 8.73300000e+03],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.32938297e-03, 2.58037140e+02, 1.11725000e+05]],

 [[2.28693771e-03, 2.57417575e+01, 1.14941986e+04],
 [5.61900595e+00, 9.36999972e+04, 4.18388830e+07],
...
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [4.60847702e-03, 4.85453461e+02, 1.42089000e+05]],

 [[2.89497635e-03, 1.67700965e+02, 6.04019202e+04],
 [6.89004371e-01, 3.99128297e+04, 1.43756570e+07],
 [3.32703483e-03, 1.75985176e+02, 6.15670000e+04],
 [8.24321432e-05, 8.12120438e-01, 2.39000000e+02],
 [3.39799356e-03, 3.92064453e+02, 1.91005000e+05]],

 [[4.16208535e-03, 2.86557806e+02, 6.89801196e+04],
 [8.69875838e-01, 5.98905814e+04, 1.44168450e+07],
 [4.60847672e-03, 2.81716187e+02, 6.67140000e+04],
 [9.78207908e-04, 5.82796707e+01, 2.87110000e+04],
 [4.60847708e-03, 6.03904053e+02, 1.33828000e+05]],

 [[2.16822023e-03, 1.06236529e+02, 5.05770076e+04],
 [5.13868195e-01, 2.79402072e+04, 1.33017530e+07],
 [2.29921378e-03, 1.13707619e+02, 5.78860000e+04],
 [8.85650581e-04, 0.00000000e+00, 0.00000000e+00],
 [4.60847706e-03, 3.44663391e+02, 1.15273000e+05]]])
Coordinates:
 * measurement (measurement) object 24B 'density' ... 'built-up volume'
 * zonal_statistics (zonal_statistics) <U6 120B 'mean' 'sum' ... 'min' 'max'
 * geometry (geometry) object 184B POLYGON ((1494534.3281858459 559...
 index (geometry) object 184B ('relation', 215618) ... ('relat...
Indexes:
 geometry GeometryIndex (crs=ESRI:54009) xarray.DataArray geometry : 23 zonal_statistics : 5 measurement : 3 0.001935 30.55 1.934e+04 4.327 ... 0.0 0.004608 344.7 1.153e+05 array([[[1.93530445e-03, 3.05514687e+01, 1.93385830e+04],
 [4.32734075e+00, 8.02281569e+04, 5.07831190e+07],
 [2.37775026e-03, 2.24578524e+01, 1.65745000e+04],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.31672247e-03, 2.00829544e+02, 1.49180000e+05]],

 [[1.55306987e-03, 1.75203519e+01, 1.24434552e+04],
 [4.03953473e+00, 5.88859028e+04, 4.18224530e+07],
 [1.70373389e-03, 9.53239155e+00, 7.51800000e+03],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.21869412e-03, 1.23922935e+02, 1.45769000e+05]],

 [[2.97495556e-03, 3.44800938e+01, 1.28181432e+04],
 [9.93932653e+00, 1.35334368e+05, 5.03112120e+07],
 [3.32938273e-03, 2.29527645e+01, 8.73300000e+03],
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [3.32938297e-03, 2.58037140e+02, 1.11725000e+05]],

 [[2.28693771e-03, 2.57417575e+01, 1.14941986e+04],
 [5.61900595e+00, 9.36999972e+04, 4.18388830e+07],
...
 [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 [4.60847702e-03, 4.85453461e+02, 1.42089000e+05]],

 [[2.89497635e-03, 1.67700965e+02, 6.04019202e+04],
 [6.89004371e-01, 3.99128297e+04, 1.43756570e+07],
 [3.32703483e-03, 1.75985176e+02, 6.15670000e+04],
 [8.24321432e-05, 8.12120438e-01, 2.39000000e+02],
 [3.39799356e-03, 3.92064453e+02, 1.91005000e+05]],

 [[4.16208535e-03, 2.86557806e+02, 6.89801196e+04],
 [8.69875838e-01, 5.98905814e+04, 1.44168450e+07],
 [4.60847672e-03, 2.81716187e+02, 6.67140000e+04],
 [9.78207908e-04, 5.82796707e+01, 2.87110000e+04],
 [4.60847708e-03, 6.03904053e+02, 1.33828000e+05]],

 [[2.16822023e-03, 1.06236529e+02, 5.05770076e+04],
 [5.13868195e-01, 2.79402072e+04, 1.33017530e+07],
 [2.29921378e-03, 1.13707619e+02, 5.78860000e+04],
 [8.85650581

You may have noticed that our data cube now has one more dimension
`zonal_statistics`, reflecting each of the aggregation methods specified
above.

> **Other statistics**
>
> The `stats` keyword is quite flexible and allows you to pass even your
> custom functions. Check the
> [documentation](https://xvec.readthedocs.io/en/stable/zonal_stats.html)
> for details.

To check the result on a map, convert the data to a
`geopandas.GeoDataFrame` again.

In [40]:
zones = zonal_stats.xvec.to_geodataframe(name="stats")
zones

345 rows × 3 columns

Check the result on a map to verify that all worked as expected. Get
mean density and explore its values, stored in the `stats` column.

In [41]:
zones.loc[("mean", 'density')].explore("stats", cmap="cividis_r", tiles="CartoDB Positron")

/tmp/ipykernel_3404/4164103204.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  zones.loc[("mean", 'density')].explore("stats", cmap="cividis_r", tiles="CartoDB Positron")

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_9e01405cbca69d8441f71da9b056ce3a {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>
 
 
 <style>
 .foliumtooltip {
 
 }
 .foliumtooltip table{
 margin: auto;
 }
 .foliumtooltip tr{
 text-align: left;
 }
 .foliumtooltip th{
 padding: 2px; padding-right: 8px;
 }
 </style>
 
 <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
</head>
<body>
 
 
 <div class="folium-map" id="map_9e01405cbca69d8441f71da9b056ce3a" ></div>
 
</body>
<script>
 
 
 var map_9e01405cbca69d8441f71da9b056ce3a = L.map(
 "map_9e01405cbca69d8441f71da9b056ce3a",
 {
 center: [47.48141835, 19.13001574999999],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 }
 );
 L.control.scale().addTo(map_9e01405cbca69d8441f71da9b056ce3a);

 

 
 
 var tile_layer_886ce5cffffc5046ec56395695c700eb = L.tileLayer(
 "https://a.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
 {"attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"https://carto.com/attributions\"\u003eCARTO\u003c/a\u003e", "detectRetina": false, "maxZoom": 20, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 );
 
 
 tile_layer_886ce5cffffc5046ec56395695c700eb.addTo(map_9e01405cbca69d8441f71da9b056ce3a);
 
 
 map_9e01405cbca69d8441f71da9b056ce3a.fitBounds(
 [[47.34968990000001, 18.925105699999992], [47.61314679999999, 19.334925799999994]],
 {}
 );
 
 
 function geo_json_61f34c561487d9aff4b6f8ad1def296c_styler(feature) {
 switch(feature.properties.index) {
 case "(\u0027relation\u0027, 215618)": 
 return {"color": "#d1bf61", "fillColor": "#d1bf61", "fillOpacity": 0.5, "weight": 2};
 case "(\u0027relation\u0027, 215621)": 
 return {"color": "#f9e03a", "fillColor": "#f9e03a", "fillOpacity": 0.5, "weight": 2};
 case "(\u0027relation\u0027, 221976)": 
 return {"color": "#707173", "fillColor": "#707173", "fillOpacity": 0.5, "weight": 2};
 case "(\u0027relation\u0027, 221980)": 
 return {"color": "#ada272", "fillColor": "#ada272", "fillOpacity": 0.5, "weight": 2};
 case "(\u0027relation\u0027, 221984)": case "(\u0027relation\u0027, 1551290)": 
 return {"color": "#928d78", "fillColor": "#928d78", "fillOpacity": 0.5, "weight": 2};
 case "(\u0027relation\u0027, 221995)": 
 return {"color"

> **Additional reading**
>
> Have a look at the chapter [*Local Spatial
> Autocorrelation*](https://geographicdata.science/book/notebooks/07_local_autocorrelation.html#bonus-local-statistics-on-surfaces)
> from the Geographic Data Science with Python by @rey2023geographic to
> learn how to do LISA on rasters.
>
> The great resource on xarray is their
> [tutorial](https://tutorial.xarray.dev/intro.html).